<a href="https://colab.research.google.com/github/darrengao628/ProtTrans/blob/master/Fine_tune_ESM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers evaluate datasets requests pandas sklearn huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached datasets-2.10.1-py3-none-any.whl (469 kB)
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
!pip evaluate

ERROR: unknown command "evaluate"


In [4]:
from huggingface_hub import notebook_login

notebook_login()

ModuleNotFoundError: ignored

In [ ]:
def data_prep(seq):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq)
    df.dropna()
    cyto=df["Subcellular location [CC]"].str.contains("Cytosol") | df["Subcellular location [CC]"].str.contains("Cytoplasm")
    mem=df["Subcellular location [CC]"].str.contains("Membrane") | df["Subcellular location [CC]"].str.contains("Cell membrane")
    cyto_df=df[cyto& ~mem]
    cyto_seq=cyto_df["Sequence"].tolist()
    cyto_labels=[0 for protein in cyto_seq]
    mem_df=df[mem& ~cyto]
    mem_seq=mem_df["Sequence"].tolist()
    mem_labels=[1 for protein in mem_seq]
    seq = cyto_seq + mem_seq
    labels=cyto_labels + mem_labels
    return seq,labels

In [ ]:
def data_prep(seq_path):# Data prepation 
    import pandas as pd
    df=pd.read_csv(seq_path)
    seq=df["seq"].tolist()
    labels=df["labels"].tolist()
    return seq,labels

In [ ]:
seq,labels=data_prep("/content/100_seq.csv")
len(labels),len(seq)

In [ ]:
def data_loader_prep (seq, labels,model_checkpoint):
    from sklearn.model_selection import train_test_split
    from transformers import AutoTokenizer
    from datasets import Dataset
    train_seq, test_seq, train_labels, test_labels = train_test_split(seq, labels, test_size =0.25,shuffle=True)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    train_tokenized=tokenizer(train_seq)
    test_tokenized=tokenizer(test_seq)
    train_ds=Dataset.from_dict(train_tokenized)
    test_ds=Dataset.from_dict(test_tokenized)
    train_ds = train_ds.add_column("labels", train_labels)
    test_ds = test_ds.add_column("labels", test_labels)
    return train_ds, train_labels, test_ds,test_labels
    

In [ ]:
model_checkpoint="facebook/esm2_t12_35M_UR50D"
train_ds, train_labels, test_ds,test_labels = data_loader_prep (seq, labels, model_checkpoint)
train_ds,test_ds 

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
num_labels = max(train_labels + test_labels) + 1  # Add 1 since 0 can be a label
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
model_name=model_checkpoint.split('/')[-1]

In [ ]:
batch_size = 8

args = TrainingArguments(
    f"{model_name}_Finetuned_localization_March_14_colab_100_seq_Epochs_50",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

from evaluate import load
import numpy as np

metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model="Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab_100_seq")

In [ ]:
import pandas as pd

In [ ]:
cytodf=pd.read_csv("/content/cyto_seq.csv")

In [ ]:
memdf=pd.read_csv("/content/mem_seq.csv")

In [ ]:
cytodf

In [ ]:
memdf

In [ ]:
seq=cytodf.loc[5,"seq"]
seq

In [ ]:
pred=pipe(seq)


In [ ]:
pred=pipe(seq)
for item in pred:
  print(item['label'])
  for d in item:
    print(d)



In [ ]:
label_0_count = 0
label_1_count = 0

for i in range(200):
  seq=cytodf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pipe(seq)
  for d in pred:
   
      if d['label'] == 'LABEL_0':
          label_0_count += 1
      elif d['label'] == 'LABEL_1':
          label_1_count += 1
print(f"LABEL_0 count: {label_0_count}")
print(f"LABEL_1 count: {label_1_count}")            
  


In [ ]:
label_0_count = 0
label_1_count = 0

for i in range(200):
  seq=memdf.loc[i,"seq"]
  pred=pipe(seq)
  for d in pred:
   
      if d['label'] == 'LABEL_0':
          label_0_count += 1
      elif d['label'] == 'LABEL_1':
          label_1_count += 1
print(f"LABEL_0 count: {label_0_count}")
print(f"LABEL_1 count: {label_1_count}")  

In [ ]:
# Test 200 seq model

# Test 200 seq model


In [ ]:
pip_200 = pipe = pipeline(model="Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_14_colab")

In [ ]:
pip_full = pipe = pipeline(model="Darren628/esm2_t12_35M_UR50D_Finetuned_localization_March_9")

In [ ]:
for i in range(5):
  seq=memdf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pip_full(seq)
  print(pred)

In [ ]:
for i in range(5):
  seq=cytodf.loc[i,"seq"]
  print(f"Prediction the {i}st seq")
  pred=pip_full(seq)
  print(pred)